In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
n = ['idx','tScore','tText']
data = pd.read_csv('/content/drive/MyDrive/ii champ/train.csv', sep=';',names=n, error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data.idx[13999]

'13998\tNegative\tСначала было все банально. Взял в кредит mp3 плеер на 6 мес. Всегда платил заранее до наступления срока платежа и, причем несколько больше, чем было предписано в договоре.Последний платеж должен был поступить от меня до 13.11.06 включительно. 23.10.2006 я пошел в ДО Красносельский с целью полностью  погасить кредит, дождался своей очереди. Меня принял операционист Д. (не буду ПОКА называть полностью его имя).Я сказал ему примерно следующее:  «Хочу полностью погасить кредит и сдать карточку (локальную, для оплаты кредита), посчитайте, сколько мне нужно разместить средств сейчас на кредитном счете, чтобы в последний день платежа по кредиту (13 ноября) у меня кредит закрылся автоматически, и я ничего не был должен банку».  Я знал, что по моему договору проценты не пересчитываются. Я просто хотел разместить сумму достаточную для полного погашения с учетом всего долга, процентов и комиссий.Без проблем. Д. мне распечатал «Справку по потребительскому кредиту» на 23.10.2006, 

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import re
import numpy as np
from tensorflow.keras.layers import Dense,SimpleRNN,Input,Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, LSTM, Input, Dropout, Embedding

text=[]
id=[]
score=[]



for i in range(len(data)):
  asa=data.idx[i].split("\t")
  id.append(asa[0])
  if asa[1]=='Positive':
    score.append([0,1])
  else:
    score.append([1,0])
  text.append(asa[2])



In [ ]:
id.pop(0)
text.pop(0)
score.pop(0)

[1, 0]

In [ ]:
score[0]

[0, 1]

In [ ]:
maxWordsCount = 20000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(text)

In [ ]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])
print(text[0][:100])

[('в', 118491), ('альфа', 7392), ('банке', 7267), ('работает', 1745), ('замечательная', 32), ('девушка', 2816), ('ильясова', 2), ('орна', 2), ('вежливая', 99), ('отзывчивая', 14)]
В Альфа-Банке работает замечательная девушка - Ильясова Орна, вежливая, отзывчивая, действительно уч


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_text_len = 150
data = tokenizer.texts_to_sequences(text)
data_pad = pad_sequences(data, maxlen=max_text_len)


In [ ]:
data_pad[2]

array([   31,    93,     8,  2966,    40,  1059,    13,     1,   554,
         339,     6,   276,    45,   178,    29,  2840,  2910, 14707,
           6,  2639,    46,    71,     2,   501,   293,   773,    36,
          60,   646,     9,   112,    96,    11,    67,     5,   177,
          27,    33,  2298,   125,     2,  1299,   154,     4,   813,
         823, 12312,  2658,    33,  5022,     8,   205, 14035,    22,
         582,     6,  4245,    43,    73,     6,  3543,  1481,   546,
          40,  3785,    25,  3937,  2446,    31,   241,    15,  1232,
         317,  2353,  1445,     8,   445,   342,   142,     9,  5296,
           2,  2353,     2,  4800,   546,    54,   497,     1,    12,
        7973,     1,   181,   181,  6493,     4,  1564,    18,    64,
         640,    45,     4, 14036,  1304,     1,  2719,   215,     6,
         197,   134,  2988, 18795,    10,  1898,   484,     2,   456,
        1216,     7, 12838,   493,    15,    19,    30,  1704,   517,
        1402,     5,

In [ ]:
X=data_pad
Y=np.array(score)

In [ ]:
X

In [ ]:
Y

In [ ]:
model = Sequential()
model.add(Embedding(maxWordsCount, 512, input_length = max_text_len))
model.add(LSTM(512, return_sequences=True))
model.add(LSTM(256))
model.add(Dense(2, activation='softmax'))
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 512)          10240000  
                                                                 
 lstm (LSTM)                 (None, 150, 512)          2099200   
                                                                 
 lstm_1 (LSTM)               (None, 256)               787456    
                                                                 
 dense (Dense)               (None, 2)                 514       
                                                                 
Total params: 13,127,170
Trainable params: 13,127,170
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

In [ ]:
history = model.fit(X, Y, batch_size=128, epochs=3)

Epoch 1/3
110/110 [==============================] - 653s 6s/step - loss: 0.4104 - accuracy: 0.8101
Epoch 2/3
110/110 [==============================] - 656s 6s/step - loss: 0.1342 - accuracy: 0.9519
Epoch 3/3
110/110 [==============================] - 652s 6s/step - loss: 0.0673 - accuracy: 0.9786


In [ ]:
n = ['idx','tScore','tText']
data = pd.read_csv('/content/drive/MyDrive/ii champ/test.csv', sep=';',names=n, error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data

In [ ]:
text=[]
id=[]


for i in range(len(data)):
  asa=data.idx[i].split("\t")
  id.append(asa[0])
  text.append(asa[1])

In [ ]:
text.pop(0)
id.pop(0)

'idx'

In [ ]:
len(text)

In [ ]:
id[0]

In [ ]:
ans=[]

In [ ]:
for i in range(len(text)):
  t = text[i].lower()
  d = tokenizer.texts_to_sequences([t])
  data_pad11 = pad_sequences(d, maxlen=max_text_len)
  res = model.predict(data_pad11)
  ans.append([id[i],np.argmax(res)])

In [ ]:
ans[:100]

In [ ]:
res = model.predict(data_pad11)
print(res, np.argmax(res), sep='\n')

In [ ]:
ans1=[]
ans1.append('idx\tScore')
for i in ans:
  if i[1]==0:
    sq='Negative'
  else:
    sq='Positive'
  ans1.append(i[0]+'\t'+sq)

In [ ]:
import csv
with open('/content/drive/MyDrive/ii champ/answer8.csv', 'w',encoding='utf-8') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL,delimiter='\n',quotechar='"')
    wr.writerow(ans1)